< [Prepare the initial section of the model](STREAM_Initial.ipynb) | [Contents](Contents.ipynb) >

# 4. Construction du modèle dynamique
Dans la section précédente, nous avons préparé la section « initiale » du modèle STREAM. C'est à dire. nous avons chargé les cartes statiques, défini les constantes et créé des rasters en entrée à l'aide de tables de recherche.

Nous allons maintenant construire le modèle dynamique dans la section `dynamic`.

## 4.1 Interception
Auparavant, nous avons déjà ajouté le code pour lire les précipitations pour chaque pas de temps.
Maintenant, nous devons calculer la quantité d'interception.
La quantité d'interception est la quantité de précipitations, mais limitée au seuil d'interception dépendant de l'utilisation des terres.

Par conséquent, nous ajoutons la ligne suivante au code :

`` `Python
Interception = min (Précipitation, self.InterceptionThreshold)
`` `

L´opération PCRaster [min](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_min.html) est utilisée ici pour obtenir la valeur minimale de `Preciptiation` et` self .InterceptionThreshold` pour chaque cellule.

Ajoutons également une ligne pour signaler la quantité d'« Interception » en tant que pile de cartes PCRaster (série de cartes temporelles).

In [1]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
         # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        
        
       # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # seuil d´interception (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # stockage maximal zone non saturée (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # coefficient de séparation (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Coefficient d'écoulement rapide (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potentiel de remontée capillaire (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Ajoutez ici le calcul de la carte de direction d´écoulement
        self.flowdirection = lddcreate(DEM,1e31,1e31,1e31,1e31)
        self.flowdirection = lddmask(self.flowdirection,self.mask)
        self.report(self.flowdirection,"./Data/ldd")
        
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

ModuleNotFoundError: No module named 'pcraster'

Maintenant nous pouvons utiliser Aguila à partir de l'invite de commande pour comparer les séries temporelles de précipitation et d'interception.
Allez à l'invite de commande, changez le répertoire pour les données et tapez à l'invite :
```
aguila --timesteps [1,10,1] pr int d
```

Avec `[1,10,1]` nous indiquons respectivement le premier pas de temps, le dernier pas de temps et l'intervalle.
Vérifiez les résultats.

## 4.2 Précipitation nette
La précipitation nette est la quantité de précipitation moins l'interception.

Ajoutez le code à la ligne 56 et utilisez NetPrecipitation comme variable de sortie. Ajoutez également une ligne pour écrire le résultat sur le disque avec le nom de fichier `pn`.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
         # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        
        
       # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # seuil d´interception (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # stockage maximal zone non saturée (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # coefficient de séparation (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Coefficient d'écoulement rapide (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potentiel de remontée capillaire (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Ajoutez ici le calcul de la carte de direction d´écoulement
        self.flowdirection = lddcreate(DEM,1e31,1e31,1e31,1e31)
        self.flowdirection = lddmask(self.flowdirection,self.mask)
        self.report(self.flowdirection,"./Data/ldd")
        
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        # Calculer les précipitations nettes

        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Exécutez le modèle et visualisez le résultat de `NetPrécipitation` en utilisant Aguila à l'invite de commande.

## 4.3 Evapotranspiration
L'évapotranspiration réelle est disponible à partir des stations météorologiques de la zone d'étude. Les emplacements des stations sont sur une carte que nous avons lue dans la section `initiale` : `self.metstat`.

Visualisez `metstat.map` avec Aguila à partir de l'invite de commande. 
* Combien de stations météorologiques y a-t-il dans la zone d'étude ?
* Quel est le type de données de cette carte ?

Les données d'évapotranspiration sont stockées dans la base de données dans [PCRaster time series](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/secdatbase.html#time-series-format) dans le fichier `et.tss`.
Jetons un coup d'oeil à ce fichier (vous pouvez l'ouvrir dans le Bloc-notes par exemple).

```
"ET,time series"			
4			
time			
station 1			
station 2			
station 3			
1	6.152	5.231	5.605
2	6.002	5.324	5.528
3	4.165	3.507	3.749
...
```

La première ligne du fichier contient le titre.
La deuxième ligne contient le nombre de colonnes.
Les lignes suivantes ont les noms des colonnes. La première est toujours `time`. Les autres correspondent aux numéros des stations dans `metstat.map` : la colonne 2 a les valeurs pour la station 1, la colonne 3 pour la station 2, etc. Chaque ligne contient les valeurs pour un pas de temps.

Avec le fichier de séries temporelles PCRaster et la carte correspondante avec les stations météorologiques, nous pouvons créer une carte avec l'évapotranspiration pour chaque station à chaque pas de temps en utilisant l'opération [`timeinput...`](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_timeinput....html). Comme nos données donnent des cartes scalaires, l'opération est `timeinputscalar`.

Nous ajoutons donc la ligne suivante au script :
```Python
ETStations = timeinputscalar("./Data/et.tss",self.metstat)
```
Nous reportons également le résultat sur le disque. Pendant le développement d'un modèle, il est recommandé de sauvegarder les résultats sur le disque pour vérifier si le résultat est conforme à nos attentes. Plus tard, nous pouvons commenter ces lignes pour rendre le calcul plus rapide.

*Notez que nous lisons maintenant la carte de direction d´écoulement depuis le disque pour gagner du temps*.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        
        # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # seuil d´interception (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # stockage maximal zone non saturée (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # coefficient de séparation (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Coefficient d'écoulement rapide (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potentiel de remontée capillaire (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
         #Calculer les précipitations nettes
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
         # Créer des cartes avec l'évapotranspiration réelle
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
                
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Exécuter le modèle et vérifier le résultat avec Aguila à partir de la ligne de commande.

Avoir l'évapotranspiration seulement aux stations météorologiques n'est pas si utile dans notre modèle spatialement explicite. Nous devons donc interpoler les points pour couvrir l'ensemble de la zone d'étude.

Auparavant, vous avez appris à interpoler en utilisant les polygones de Thiessen et IDW. Ajoutez le code à la ligne 60 et appelez la sortie `ET`. A la ligne 61, reportez le résultat sur le disque.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        
        # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # seuil d´interception (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # stockage maximal zone non saturée (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # coefficient de séparation (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Coefficient d'écoulement rapide (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potentiel de remontée capillaire (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
         #Calculer les précipitations nettes
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
         # Créer des cartes avec l'évapotranspiration réelle
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpoler ETa avec IDW

        
                
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Exécutez le modèle et vérifiez le résultat avec Aguila à partir de l'invite de commande.

* Pouvez-vous modifier le code pour faire l'interpolation avec les polygones de Thiessen ?
* Quelle serait la meilleure solution à utiliser compte tenu de la quantité et de la répartition des stations météorologiques ?

## 4.4 Zone non saturée

Ensuite, la précipitation nette est séparée en infiltration dans la zone non saturée et saturée en utilisant un coefficient de séparation.
Dans la section `initiale`, nous avons déjà créé une carte `SeparationCoefficient` en utilisant un tableau de correspondance avec la carte d'occupation des sols. Nous avons également défini un stockage initial de l'humidité du sol `self.Su` (mm). Pour calculer le nouveau stockage d'humidité du sol, nous ajoutons le code suivant :

```Python
self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
```

La zone non saturée est remplie jusqu'à ce que la capacité du champ, `SuMax` (mm) soit atteinte. L'excès d'humidité du sol, `SuExcess` (mm) va vers la zone saturée avec un facteur de retard de `Ku`. C'est le seul échange entre la zone non saturée et la zone saturée. En code :

```Python
SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
```

Ensuite, le nouveau stockage de l'humidité du sol dans la zone non saturée peut être calculé :

```Python
self.Su = self.Su - SuExcess
```

Et nous devons également soustraire l'évapotranspiration :

```Python
self.Su = self.Su - ET
```

Ajoutons le code aux lignes 64-68 et écrivons le résultat sur le disque.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        
        # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # seuil d´interception (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # stockage maximal zone non saturée (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # coefficient de séparation (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Coefficient d'écoulement rapide (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potentiel de remontée capillaire (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
         #Calculer les précipitations nettes
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
         # Créer des cartes avec l'évapotranspiration réelle
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpoler ETa avec IDW

        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modélisation de la zone non saturée

        
        
        
        
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Vérifiez le résultat en visualisant `su` avec Aguila à partir de l'invite de commande.

## 4.5 Zone saturée

La zone saturée peut être modélisée comme la somme du stockage de la zone saturée, de la quantité de précipitations nettes qui s'infiltrent et de l'excès de la zone non saturée :

```Python
self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
```

Ajoutons ceci au script à la ligne 71. Rapportez le résultat à la ligne 72.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        
        # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # seuil d´interception (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # stockage maximal zone non saturée (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # coefficient de séparation (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Coefficient d'écoulement rapide (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potentiel de remontée capillaire (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
         #Calculer les précipitations nettes
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
         # Créer des cartes avec l'évapotranspiration réelle
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpoler ETa avec IDW

        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modélisation de la zone non saturée

        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modélisation de la zone saturée

              
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Exécutez le modèle et vérifiez le résultat avec Aguila dans l'invite de commande.

## 4.6 Écoulement de surface saturé
Il y a cependant un maximum au stockage de la zone saturée `SsMax` (mm). L'écoulement souterrain saturé se produit lorsque le stockage de la zone saturée `Ss` dépasse `SsMax`. Cela se produit normalement lorsque la nappe phréatique atteint la surface dans des conditions de sol saturé. La situation dans laquelle la nappe phréatique atteint la surface est déterminée par le DEM et le fond de la rivière drainante `Ss0` (m). `SsDEM` (m) est défini comme la distance entre la surface et le fond de la rivière drainante.
```Python
SsDEM = DEM + self.Ss0
```

En raison de la porosité, `SsDEM` n'est pas complètement disponible pour l'eau. Nous utilisons donc la relation empirique suivante :
```Python
self.SsMax = 25 * ln(SsDEM)
```

Ces équations ne sont pas dépendantes du temps. Elles doivent donc être ajoutées à la section `initial`. Ajoutez le code aux lignes 47 et 48 et reportez le résultat à la ligne 49.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        
        # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # seuil d´interception (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # stockage maximal zone non saturée (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # coefficient de séparation (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Coefficient d'écoulement rapide (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potentiel de remontée capillaire (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Calculer le maximum au stockage de la zone saturée

        
        
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculer les précipitations nettes
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
         # Créer des cartes avec l'évapotranspiration réelle
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpoler ETa avec IDW

        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modélisation de la zone non saturée

        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modélisation de la zone saturée
        self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
        self.report(self.Ss,"./Data/ss")
        
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Maintenant, nous pouvons calculer le débit de surface saturé comme la quantité qui dépasse le stockage de la zone saturée:
```Python
SaturatedOverlandFlow = ifthenelse(self.Ss > self.SsMax, self.Ss - self.SsMax, 0)
```
Cette condition se lit comme suit : si le stockage de la zone saturée est supérieur au stockage maximum, alors le débit de surface saturé est égal à la différence entre le stockage de la zone saturée et le stockage maximum. Sinon, le débit de la zone saturée est égal à zéro.

Après avoir calculé le montant du débit de la zone saturée, nous devons mettre à jour le stockage de la zone saturée :
```Python
self.Ss = self.Ss - SaturatedOverlandFlow
```

Ajoutons ceci au script en lignes 81 et 82. Rapportez le résultat à la ligne 83.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        
        # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # seuil d´interception (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # stockage maximal zone non saturée (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # coefficient de séparation (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Coefficient d'écoulement rapide (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potentiel de remontée capillaire (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Calculer le maximum au stockage de la zone saturée
        SsDEM = DEM + self.Ss0
        self.SsMax = 25 * ln(SsDEM)
        self.report(self.SsMax,"./Data/ssmax")
        
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculer les précipitations nettes
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
         # Créer des cartes avec l'évapotranspiration réelle
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpoler ETa avec IDW

        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modélisation de la zone non saturée

        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modélisation de la zone saturée
        self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
        self.report(self.Ss,"./Data/ss")
        
        
        # Calculer l´écoulement de surface saturé

        
        
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Exécutez le modèle et vérifiez le résultat avec Aguila dans l'invite de commande.

## 4.7 Écoulement rapide
La composante d'écoulement rapide des eaux souterraines est l'écoulement des eaux souterraines à travers les macropores et les fissures. Dans le modèle STREAM, nous calculons le seuil pour le flux rapide `SsQuick` en multipliant le stockage maximal de la zone saturée `self.SsMax` avec le coefficient d'écoulement rapide `self.QuickFlowCoefficient`, que nous avons dérivé de la carte des sols et d'une table de correspondance dans la section `initial`.
```Python
SsQuick = self.SsMax * self.QuickFlowCoefficient
```
Ensuite, l´écoulement rapide réel peut être calculé comme suit :
```Python
QuickFlow = max((self.Ss - SsQuick), 0) / self.rtq
```
`self.rtq` est la constante de récession de l´écoulement rapide que nous avons déjà défini dans la section `initial`.

Enfin, nous devons également mettre à jour le stockage de la zone saturée en supprimant l´écoulement rapide :
```Python
self.Ss = self.Ss - QuickFlow
```

Ajoutez ceci au code ci-dessous à la ligne 86 et plus loin. Rapportez ce qui est nécessaire.


In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        
        # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # seuil d´interception (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # stockage maximal zone non saturée (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # coefficient de séparation (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Coefficient d'écoulement rapide (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potentiel de remontée capillaire (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Calculer le maximum au stockage de la zone saturée
        SsDEM = DEM + self.Ss0
        self.SsMax = 25 * ln(SsDEM)
        self.report(self.SsMax,"./Data/ssmax")
        
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculer les précipitations nettes
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
         # Créer des cartes avec l'évapotranspiration réelle
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpoler ETa avec IDW

        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modélisation de la zone non saturée

        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modélisation de la zone saturée
        self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
        self.report(self.Ss,"./Data/ss")
        
        
        # Calculer l´écoulement de surface saturé

        SaturatedOverlandFlow = ifthenelse(self.Ss > self.SsMax, self.Ss - self.SsMax, 0)
        self.Ss = self.Ss - SaturatedOverlandFlow
        self.report(SaturatedOverlandFlow,"./Data/saof")
        
        # Calculer l'écoulement rapide
        
        
        
                             
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

## 4.8 Ecoulement lent
L´écoulement lent dépend du stockage dans la zone saturée (`self.Ss`) et est divisé par le coefficient de récession pour le débit lent (`self.rts`) que nous avons défini précédemment dans la section `initial` :
```Python
SlowFlow = max(self.Ss, 0) / self.rts
```
Ici aussi, nous devons mettre à jour le stockage de la zone saturée :
```Python
self.Ss = self.Ss - SlowFlow
```
Écrivons également le `SlowFlow` sur le disque.
Ajoutez ces lignes au code à la ligne 92.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        
        # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # seuil d´interception (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # stockage maximal zone non saturée (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # coefficient de séparation (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Coefficient d'écoulement rapide (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potentiel de remontée capillaire (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Calculer le maximum au stockage de la zone saturée
        SsDEM = DEM + self.Ss0
        self.SsMax = 25 * ln(SsDEM)
        self.report(self.SsMax,"./Data/ssmax")
        
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculer les précipitations nettes
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
         # Créer des cartes avec l'évapotranspiration réelle
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpoler ETa avec IDW
        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modélisation de la zone non saturée
        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modélisation de la zone saturée
        self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
        self.report(self.Ss,"./Data/ss")
        
        
        # Calculer l´écoulement de surface saturé
        SaturatedOverlandFlow = ifthenelse(self.Ss > self.SsMax, self.Ss - self.SsMax, 0)
        self.Ss = self.Ss - SaturatedOverlandFlow
        self.report(SaturatedOverlandFlow,"./Data/saof")
        
        # Calculer l'écoulement rapide
        SsQuick = self.SsMax * self.QuickFlowCoefficient
        QuickFlow = max((self.Ss - SsQuick), 0) / self.rtq
        self.Ss = self.Ss - QuickFlow
        self.report(QuickFlow, "./Data/qflo")  
        
        # Calculer l'écoulement lent

        
        
        
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

## 4.9 Remontée capillaire
La remontée capillaire de la zone saturée vers la zone non saturée est définie par une valeur minimale et maximale.
La valeur minimale `MinCapRise` est un quart du stockage maximal de la zone saturée.

Ajoutez ceci à la ligne 52 de la section `initial` ci-dessous, car elle est indépendante du temps :

```Python
self.MinCapRise = self.SsMax / 4.0
```

Si le stockage de la zone saturée est supérieur à la remontée capillaire minimale, alors la remontée capillaire est le minimum de la quantité disponible dans le stockage de la zone saturée, de l'évapotranspiration et de la remontée capillaire maximale :
```Python
CapRise = ifthenelse(self.Ss > self.MinCapRise, min(self.MaxCapRise,ET,self.Ss), self.MinCapRise)
```
Ajoutez ceci à la ligne 100 ci-dessous.

Maintenant, nous devons également mettre à jour les stockages des zones saturées et non saturées.


```Python
self.Ss = self.Ss - CapRise
self.Su = self.Su + CapRise
```

Ajoutez ceci aux lignes 101 et 102 ci-dessous et rapportez `self.Ss` et `self.Su`. 
Exécutez le modèle ci-dessous et vérifiez les résultats.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        
        # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # seuil d´interception (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # stockage maximal zone non saturée (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # coefficient de séparation (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Coefficient d'écoulement rapide (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potentiel de remontée capillaire (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Calculer le maximum au stockage de la zone saturée
        SsDEM = DEM + self.Ss0
        self.SsMax = 25 * ln(SsDEM)
        self.report(self.SsMax,"./Data/ssmax")
        
        
        # Remontée capillaire minimale

        
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculer les précipitations nettes
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
         # Créer des cartes avec l'évapotranspiration réelle
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpoler ETa avec IDW
        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modélisation de la zone non saturée
        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modélisation de la zone saturée
        self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
        self.report(self.Ss,"./Data/ss")
        
        
        # Calculer l´écoulement de surface saturé
        SaturatedOverlandFlow = ifthenelse(self.Ss > self.SsMax, self.Ss - self.SsMax, 0)
        self.Ss = self.Ss - SaturatedOverlandFlow
        self.report(SaturatedOverlandFlow,"./Data/saof")
        
        # Calculer l'écoulement rapide
        SsQuick = self.SsMax * self.QuickFlowCoefficient
        QuickFlow = max((self.Ss - SsQuick), 0) / self.rtq
        self.Ss = self.Ss - QuickFlow
        self.report(QuickFlow, "./Data/qflo")  
        
        # Calculer l'écoulement lent
        SlowFlow = max(self.Ss, 0) / self.rts
        self.Ss = self.Ss - SlowFlow
        self.report(SlowFlow,"./Data/sflo")
        
        # Remontée capillaire

        
        
        
        
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

## 4.10 Modélisation des écoulements
Le ruissellement total est la somme de l'écoulement de surface saturé, de l'écoulement rapide et de l'écoulement lent :
```Python
Runoff = SaturatedOverlandFlow + QuickFlow + SlowFlow
```
Avec l'opération PCRaster [accuflux](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_accuflux.html) nous pouvons accumuler la quantité de ruissellement sur la carte de direction du flux et calculer le débit :
```Python
Discharge = accuflux(self.flowdirection, Runoff) * self.ConvConst
```
Cette approche peut être utilisée lorsque l'on peut supposer que toute l'eau atteint la sortie en un seul pas de temps.
Notez que nous multiplions avec un facteur de conversion pour avoir les unités correctes.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        
        # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # seuil d´interception (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # stockage maximal zone non saturée (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # coefficient de séparation (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Coefficient d'écoulement rapide (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potentiel de remontée capillaire (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Calculer le maximum au stockage de la zone saturée
        SsDEM = DEM + self.Ss0
        self.SsMax = 25 * ln(SsDEM)
        self.report(self.SsMax,"./Data/ssmax")
        
        
        # Remontée capillaire minimale
        self.MinCapRise = self.SsMax / 4.0
        
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculer les précipitations nettes
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
         # Créer des cartes avec l'évapotranspiration réelle
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpoler ETa avec IDW
        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modélisation de la zone non saturée
        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modélisation de la zone saturée
        self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
        self.report(self.Ss,"./Data/ss")
        
        
        # Calculer l´écoulement de surface saturé
        SaturatedOverlandFlow = ifthenelse(self.Ss > self.SsMax, self.Ss - self.SsMax, 0)
        self.Ss = self.Ss - SaturatedOverlandFlow
        self.report(SaturatedOverlandFlow,"./Data/saof")
        
        # Calculer l'écoulement rapide
        SsQuick = self.SsMax * self.QuickFlowCoefficient
        QuickFlow = max((self.Ss - SsQuick), 0) / self.rtq
        self.Ss = self.Ss - QuickFlow
        self.report(QuickFlow, "./Data/qflo")  
        
        # Calculer l'écoulement lent
        SlowFlow = max(self.Ss, 0) / self.rts
        self.Ss = self.Ss - SlowFlow
        self.report(SlowFlow,"./Data/sflo")
        
        # Remontée capillaire
        CapRise = ifthenelse(self.Ss > self.MinCapRise, min(self.MaxCapRise,ET,self.Ss), self.MinCapRise)
        self.Ss = self.Ss - CapRise
        self.Su = self.Su + CapRise
        self.report(self.Su,"./Data/su")
        self.report(self.Ss,"./Data/ss")
        
        #Calculer le ruissellement total et le débit
        Runoff = SaturatedOverlandFlow + QuickFlow + SlowFlow
        Discharge = accuflux(self.flowdirection, Runoff) * self.ConvConst
        self.report(Discharge,"./Data/q")
        self.report(Runoff,"./Data/runoff")
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Visualisez le résultat avec Aguila à partir de l'invite de commande :
```
aguila --timesteps [1,10,1] q pr
```
Avec aguila, vous pouvez également visualiser les graphiques d'un pixel sélectionné. Cliquez à droite sur la légende et choisissez *Show time series...*.
* Y a-t-il une relation entre les précipitations et le débit des rivières ?

## Calibrage et évaluation
Après avoir créé le modèle, vous souhaitez comparer les résultats avec les mesures afin (1) de calibrer les paramètres du modèle et (2) d'évaluer les performances du modèle.

Avec PCRaster, nous pouvons créer des tableaux de séries temporelles à des points sélectionnés. Plus tôt, nous avons utilisé `col2map` pour convertir les coordonnées des lieux en une carte. Ici, nous avons créé une carte avec quatre emplacements de mesure à des points intéressants de la rivière.

Utilisez Aguila à l'invite de commande pour visualiser `measurements.map` avec `q` pour trouver où sont les points.

Pour rapporter le débit à ces points, nous devons l'initialiser dans la section `initial` du modèle en ajoutant ces trois lignes :
```Python
self.Measurements = self.readmap("./Data/measurements") # lire la carte avec les emplacements des mesures
DischargeAtMeasurementLocations = "./Data/discharge.tss" # définir le nom du fichier de sortie de la série chronologique
self.DischargeTSS = TimeoutputTimeseries(DischargeAtMeasurementLocations, self, self.Measurements, noHeader=False)
```

Dans la section `dynamique`, nous pouvons ensuite rapporter les valeurs en ajoutant cette ligne :
```Python
self.DischargeTSS.sample(Discharge)
```
Où `self.DischargeTSS` a été défini dans la section `initial` et nous utilisons `Discharge` entre parenthèses pour rapporter les valeurs de décharge.

Le code devrait maintenant ressembler à ceci :

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        self.flowdirection = self.readmap("./Data/ldd") 
        
        
        # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # seuil d´interception (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # stockage maximal zone non saturée (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # coefficient de séparation (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Coefficient d'écoulement rapide (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potentiel de remontée capillaire (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Calculer le maximum au stockage de la zone saturée
        SsDEM = DEM + self.Ss0
        self.SsMax = 25 * ln(SsDEM)
        self.report(self.SsMax,"./Data/ssmax")
        
        
        # Remontée capillaire minimale
        self.MinCapRise = self.SsMax / 4.0
        
        self.MinCapRise = self.SsMax / 4.0
        
        # initialisation de la sortie de la série temporelle
        self.Measurements = self.readmap("./Data/measurements") # lire la carte avec les emplacements des mesures
        DischargeAtMeasurementLocations = "./Data/discharge.tss" # dDéfinir le nom du fichier de sortie de la série chronologique
        self.DischargeTSS = TimeoutputTimeseries(DischargeAtMeasurementLocations, 
                                                 self, 
                                                 self.Measurements, 
                                                 noHeader=False)
    
     def dynamic(self):
        Precipitation = self.readmap("./Data/pr")
        Interception = min(Precipitation, self.InterceptionThreshold)
        self.report(Interception,"./Data/int")
        
        #Calculer les précipitations nettes
        NetPrecipitation = Precipitation - Interception
        self.report(NetPrecipitation,"./Data/pn")
        
         # Créer des cartes avec l'évapotranspiration réelle
        ETStations = timeinputscalar("./Data/et.tss",self.metstat)
        self.report(ETStations,"./Data/etstat")
        
        # Interpoler ETa avec IDW
        ET = inversedistance(self.mask,ETStations,2,0,0)
        self.report(ET,"./Data/et")   
        
        # Modélisation de la zone non saturée
        self.Su = self.Su + (1 - self.SeparationCoefficient) * NetPrecipitation
        SuExcess = max(0, ((self.Su - self.SuMax) / self.Ku))
        self.Su = self.Su - SuExcess
        self.Su = self.Su - ET
        self.report(self.Su,"./Data/su")
        
        # Modélisation de la zone saturée
        self.Ss = self.Ss + (self.SeparationCoefficient * NetPrecipitation) + SuExcess
        self.report(self.Ss,"./Data/ss")
        
        
        # Calculer l´écoulement de surface saturé
        SaturatedOverlandFlow = ifthenelse(self.Ss > self.SsMax, self.Ss - self.SsMax, 0)
        self.Ss = self.Ss - SaturatedOverlandFlow
        self.report(SaturatedOverlandFlow,"./Data/saof")
        
        # Calculer l'écoulement rapide
        SsQuick = self.SsMax * self.QuickFlowCoefficient
        QuickFlow = max((self.Ss - SsQuick), 0) / self.rtq
        self.Ss = self.Ss - QuickFlow
        self.report(QuickFlow, "./Data/qflo")  
        
        # Calculer l'écoulement lent
        SlowFlow = max(self.Ss, 0) / self.rts
        self.Ss = self.Ss - SlowFlow
        self.report(SlowFlow,"./Data/sflo")
        
        # Remontée capillaire
        CapRise = ifthenelse(self.Ss > self.MinCapRise, min(self.MaxCapRise,ET,self.Ss), self.MinCapRise)
        self.Ss = self.Ss - CapRise
        self.Su = self.Su + CapRise
        self.report(self.Su,"./Data/su")
        self.report(self.Ss,"./Data/ss")
        
        #Calculer le ruissellement total et le débit
        Runoff = SaturatedOverlandFlow + QuickFlow + SlowFlow
        Discharge = accuflux(self.flowdirection, Runoff) * self.ConvConst
        self.report(Discharge,"./Data/q")
        self.report(Runoff,"./Data/runoff")
        
        # Raport sur les séries chronologiques de débit aux emplacements de mesure
        self.DischargeTSS.sample(Discharge)
                     
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Nous pouvons utiliser Aguila à partir de l'invite de commande pour visualiser `discharge.tss` :
```
aguila discharge.tss
```
Notez que `discharge.tss` est un fichier texte que vous pouvez ouvrir dans un tableur par exemple. Aguila interprète le texte et crée les graphiques.

* Quel est l'affluent qui a le débit le plus élevé ?

Le calibrage dépasse le cadre de ce tutoriel, mais vous pouvez jouer avec le code pour voir ce qui se passe si vous changez les paramètres ou les variables. Par exemple, que se passe-t-il avec le débit si on double les précipitations ?

< [Prepare the initial section of the model](STREAM_Initial.ipynb) | [Contents](Contents.ipynb) >